In [1]:
%pip install -U huggingface_hub
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U ipywidgets
%pip install huggingface_hub[hf_xet]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 34.7 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.4
    Uninstalling huggingface-hub-0.33.4:
      Successfully uninstalled huggingface-hub-0.33.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━

In [6]:
from huggingface_hub import login
import os

hf_token = os.environ.get("HF_TOKEN")
login(hf_token)

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_dir = "Qwen/Qwen3-4B"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
train_prompt_style = """Ci-dessous se trouve une instruction qui décrit une tâche, accompagnée d’un contexte supplémentaire.
Rédige une réponse qui complète correctement la demande.
Avant de répondre, réfléchis attentivement à la question et construis une chaîne de raisonnement étape par étape pour garantir une réponse logique et précise.

### Instruction :
Tu es un robot doté de comportement très proche de l'homme et d'une intelligence le surpassant.
Merci de répondre à la question suivante en réfléchissant étape par étape.

### Question :
{}

### Réponse :
<think>
{}
</think>
{}"""


In [9]:
# Vérifier si le fichier existe bien et si il est bien formatté
import os

def check_file_format(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Le fichier {file_path} n'existe pas.")

    with open(file_path, "r", encoding="utf-8" ) as file:
        lines = file.readlines()
        if len(lines) < 3:
            raise ValueError(f"Le fichier {file_path} doit contenir au moins trois lignes.")

        for i, line in enumerate(lines):
            if not line.strip():
                raise ValueError(f"La ligne {i + 1} du fichier {file_path} est vide ou ne contient que des espaces.")
    print(f"Le fichier {file_path} est correctement formaté.")

# Chemin vers le fichier à vérifier
file_path = "../datasets/global_com.json"
check_file_format(file_path)

FileNotFoundError: Le fichier ../datasets/global_com.json n'existe pas.

In [10]:
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    complex_cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for question, cot, response in zip(inputs, complex_cots, outputs):
        if not response.endswith(tokenizer.eos_token):
            response += tokenizer.eos_token
        text = train_prompt_style.format(question, cot, response)
        texts.append(text)
    return {"text": texts}

In [11]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="./global_com.json", split="train")
dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)
dataset["text"][10]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/331 [00:00<?, ? examples/s]

"Ci-dessous se trouve une instruction qui décrit une tâche, accompagnée d’un contexte supplémentaire. \nRédige une réponse qui complète correctement la demande. \nAvant de répondre, réfléchis attentivement à la question et construis une chaîne de raisonnement étape par étape pour garantir une réponse logique et précise.\n\n### Instruction :\nTu es un robot doté de comportement très proche de l'homme et d'une intelligence le surpassant.\nMerci de répondre à la question suivante en réfléchissant étape par étape. \n\n### Question :\nAash, raconte-moi une blague !\n\n### Réponse :\n<think>\n1. L'utilisateur veut une interaction ludique, je dois raconter une blague simple. 2. Une animation de rire avec des secousses saccadées est idéale. 3. La blague doit être légère et adaptée à tous. 4. Je peux mentionner Samuel pour personnaliser.\n</think>\n[Action: Rire] Pourquoi l'ordinateur de Samuel va-t-il à l'école ? Parce qu'il veut devenir plus intelligent, comme Aash ! Haha, t'en veux une autre

In [12]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [13]:
inference_prompt_style = """Ci-dessous se trouve une instruction qui décrit une tâche, accompagnée d’un contexte supplémentaire.
Rédige une réponse qui complète correctement la demande.
Avant de répondre, réfléchis attentivement à la question et construis une chaîne de raisonnement étape par étape pour garantir une réponse logique et précise.

### Instruction :
Tu es un robot doté de comportement très proche de l'homme et d'une intelligence le surpassant.
Merci de répondre à la question suivante en réfléchissant étape par étape.

### Question :
{}

### Réponse :
<think>"""


In [14]:
!nvidia-smi
import torch
print(torch.cuda.is_available())

Sun Jul 27 20:20:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   77C    P0             34W /   70W |    4014MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [20]:
question = dataset[10]['Question']
inputs = tokenizer(
    [inference_prompt_style.format(question) + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0])
print(response[0].split("### Réponse :")[1])

Ci-dessous se trouve une instruction qui décrit une tâche, accompagnée d’un contexte supplémentaire. 
Rédige une réponse qui complète correctement la demande. 
Avant de répondre, réfléchis attentivement à la question et construis une chaîne de raisonnement étape par étape pour garantir une réponse logique et précise.

### Instruction :
Tu es un robot doté de comportement très proche de l'homme et d'une intelligence le surpassant.
Merci de répondre à la question suivante en réfléchissant étape par étape. 

### Question :
Aash, raconte-moi une blague !

### Réponse :
<think>


</think>

**Étape 1 : Comprendre la demande**  
La question demandée est une blague, et le nom de la personne est Aash. Il s'agit donc de raconter une blague, probablement en français, et de la faire comprendre à Aash.

**Étape 2 : Choisir une blague adaptée**  
Il s'agit de choisir une blague qui soit drôle, simple, et qui s'adapte à un public général. La blague doit être courte, claire, et avoir une punchline qui

IndexError: list index out of range

In [21]:
from peft import LoraConfig, get_peft_model

# LoRA config
peft_config = LoraConfig(
    lora_alpha=16,                           # Scaling factor for LoRA
    lora_dropout=0.05,                       # Add slight dropout for regularization
    r=64,                                    # Rank of the LoRA update matrices
    bias="none",                             # No bias reparameterization
    task_type="CAUSAL_LM",                   # Task type: Causal Language Modeling
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Target modules for LoRA
)

model = get_peft_model(model, peft_config)

/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [24]:
from trl import SFTTrainer
from transformers import TrainingArguments


# Training Arguments
training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    logging_steps=0.2,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none"
)

# Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    peft_config=peft_config,
    data_collator=data_collator,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Adding EOS to train dataset:   0%|          | 0/331 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/331 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/331 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [25]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

Step,Training Loss
34,1.400300
68,0.646300
102,0.552500
136,0.521600


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=166, training_loss=0.7285329749785274, metrics={'train_runtime': 1110.4096, 'train_samples_per_second': 0.298, 'train_steps_per_second': 0.149, 'total_flos': 2427093248019456.0, 'train_loss': 0.7285329749785274})

In [31]:
question = dataset[10]['Question']
inputs = tokenizer(
    [inference_prompt_style.format(question, "") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0].split("### Réponse :")[1])


<think>

1. L'utilisateur cherche un moment de rire et de joie. 2. Une animation de 'Rire' avec des mouvements légers et joyeux convient. 3. Je dois raconter une blague simple et drôle. 4. La réponse doit être amusante et engageante.
</think>
[Action: Rire] Pourquoi les mathématiciens sont-ils toujours en retard ? Parce qu'ils ont toujours des problèmes !


In [ ]:
question = dataset[100]['Question']
inputs = tokenizer(
    [inference_prompt_style.format(question, "") + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0].split("### Réponse :")[1])

In [27]:
new_model_name = "Aash"
model.push_to_hub(new_model_name)
tokenizer.push_to_hub(new_model_name)

adapter_model.safetensors:   0%|          | 0.00/529M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/punchnox/Aash/commit/b17c8d5a4fac112ee59995e317661e4fb0fcacbb', commit_message='Upload tokenizer', commit_description='', oid='b17c8d5a4fac112ee59995e317661e4fb0fcacbb', pr_url=None, repo_url=RepoUrl('https://huggingface.co/punchnox/Aash', endpoint='https://huggingface.co', repo_type='model', repo_id='punchnox/Aash'), pr_revision=None, pr_num=None)

In [23]:
import os
os.environ["WANDB_DISABLED"] = "true"